In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [14]:
df = pd.read_csv("SmallCustomers.csv")

In [15]:
print(df.head())
print(df.info())
print(df.describe())

   CustomerID  Frequency  Monetary  Recency  labels
0       12821        6.0     92.72    215.0     1.0
1       12823        5.0   1759.50     75.0     1.0
2       12829       11.0    293.00    337.0     1.0
3       12831        9.0    215.05    263.0     1.0
4       12833       24.0    417.38    146.0     1.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328 entries, 0 to 1327
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  1328 non-null   int64  
 1   Frequency   1328 non-null   float64
 2   Monetary    1328 non-null   float64
 3   Recency     1328 non-null   float64
 4   labels      1328 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 52.0 KB
None
         CustomerID    Frequency     Monetary      Recency  labels
count   1328.000000  1328.000000  1328.000000  1328.000000  1328.0
mean   15580.122741    14.237952   284.286785   163.052711     1.0
std     1609.768333    10.728576   260

In [16]:
sum(df.Frequency)

18908.0

In [17]:
#import initial filtered data to get item info
items=pd.read_csv('OnlineRetail_Filtered.csv')

In [19]:
#get small customers out of filtered data
items=items[items['CustomerID'].isin(df['CustomerID'])]

In [24]:
items.info()
items['Description'] = items['Description'].str.strip()
items['InvoiceNo'] = items['InvoiceNo'].astype(str)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18908 entries, 70 to 337808
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    18908 non-null  int64  
 1   StockCode    18908 non-null  object 
 2   Description  18908 non-null  object 
 3   Quantity     18908 non-null  int64  
 4   InvoiceDate  18908 non-null  object 
 5   UnitPrice    18908 non-null  float64
 6   CustomerID   18908 non-null  int64  
 7   Country      18908 non-null  object 
 8   Revenue      18908 non-null  float64
 9   InvoiceDay   18908 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 1.6+ MB


In [31]:
#agg by invoice number
#group by invoice no and description
basket=items.groupby(by=['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0)

In [36]:
basket.drop('POSTAGE',axis=1,inplace=True)

In [41]:
basket.set_index('InvoiceNo',inplace=True)
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
538368,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
#encode as 1 as long as item purchased
basket_sets=basket.applymap(lambda x: 1 if x>0 else 0)

# Apply Apriori algorithm.

In [62]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [63]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [64]:
rules[ (rules['lift'] >= 6) &
        (rules['confidence'] >= 0.8) ].sort_values(['lift','confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
38,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.016375,0.031056,0.014681,0.896552,28.868966,0.014172,9.366460
8,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.024280,0.031056,0.019763,0.813953,26.209302,0.019009,5.208075
33,"(PINK REGENCY TEACUP AND SAUCER, REGENCY CAKES...",(GREEN REGENCY TEACUP AND SAUCER),0.014681,0.031056,0.011858,0.807692,26.007692,0.011402,5.038509
0,(PAINTED METAL PEARS ASSORTED),(ASSORTED COLOUR BIRD ORNAMENT),0.023715,0.050254,0.019198,0.809524,16.108614,0.018006,4.986166


,support,itemsets
0,0.010164,(12 MESSAGE CARDS WITH ENVELOPES)
1,0.010728,(3 HOOK PHOTO SHELF ANTIQUE WHITE)
2,0.011858,(3 STRIPEY MICE FELTCRAFT)
3,0.014116,(4 TRADITIONAL SPINNING TOPS)
4,0.025409,(6 RIBBONS RUSTIC CHARM)
...,...,...
219,0.011293,"(WOODEN PICTURE FRAME WHITE FINISH, WOODEN FRA..."
220,0.011858,"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY..."
221,0.014681,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY..."
222,0.011858,"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC..."
